<a href="https://colab.research.google.com/github/daisywill/DeepTraffic/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip list

In [ ]:
!pip install albumentations
!pip install segmentation_models
!pip install tensorflow==1.14
!pip install keras==2.2.4
import pandas as pd
import numpy as np
import albumentations as A
import segmentation_models as sm
import os

In [5]:
import cv2
import keras
import numpy as np
import pandas as pd
import albumentations as A
import segmentation_models as sm
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras import applications
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Activation,Dropout,Flatten,Dense
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from keras.optimizers import SGD,Adam
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
import os
warnings.filterwarnings('ignore')

In [6]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(directory='/content/drive/MyDrive/dataset/PlantVillage植物叶片病害数据集',
                                                    target_size=(224, 224), 
                                                    classes=os.listdir('/content/drive/MyDrive/dataset/PlantVillage植物叶片病害数据集'))
#test_datagen = ImageDataGenerator()

Found 54330 images belonging to 38 classes.


In [7]:
resnet = keras.applications.ResNet50(weights="imagenet",include_top=False,input_shape=(224,224,3))
top_model = Sequential()
top_model.add(Flatten(input_shape=resnet.get_output_shape_at(-1)[1:]))
top_model.add(Dense(256,activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(38, activation='softmax'))
model = Sequential()
model.add(resnet)
model.add(top_model)








94658560/94653016 [==============================] - 5s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
sequential_1 (Sequential)    (None, 38)                25700134  
Total params: 49,287,846
Trainable params: 49,234,726
Non-trainable params: 53,120
_________________________________________________________________


In [9]:
import math
from keras.callbacks import ModelCheckpoint, LearningRateScheduler,ReduceLROnPlateau
def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.5
    epochs_drop = 2
    lrate = initial_lrate * math.pow(drop,
           math.floor((1+epoch)/epochs_drop))
    print(lrate)
    return lrate
lrate = LearningRateScheduler(step_decay)

callbacks = [
    keras.callbacks.ReduceLROnPlateau(),lrate
]


In [ ]:
smooth = 1.
from keras import backend as K
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(K.epsilon()+pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0 + K.epsilon()))
    return focal_loss_fixed

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
 
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

model.compile(optimizer=Adam(lr=1e-4),loss=focal_loss(alpha=0.1, gamma=2),metrics=[dice_coef,'accuracy'])
model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=4, callbacks=callbacks)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/4
0.0001
  56/1698 [..............................] - ETA: 14:56:05 - loss: 12.8082 - dice_coef: 0.1525 - acc: 0.2383

In [ ]:
model.save('/content/drive/MyDrive/dataset/classification.h5')